In [13]:
import pipelines
from stories import get_useful_projects
import os
import sys
import psycopg2
import pandas as pd
from yaml import load, Loader
import openai
from datetime import datetime
from pipelines import *

In [14]:
with open("config.yml","r") as config:
    data = load(config, Loader=Loader)
    api_key = data["apikey"]
    openai.api_key  = api_key

In [15]:
projects = get_useful_projects()
for p in projects:
    print(p['story'])

creek
minotaur
seeknumbers
sudoku
yosenabe
hop
lights


In [16]:
conn_string = "dbname=thesis user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


In [17]:
def search_encoding(conn,story,story_1,story_2,version):
    sql = "SELECT id, encoding FROM experiments WHERE story = '"+story +"' AND story_1 = '" +story_1 + "' AND story_2 = '"+ story_2 +"' AND version = "+str(version) 
    #print(sql)
    cursor = conn.cursor()
    try:
        cursor.execute(sql)
        result = cursor.fetchall()
        conn.commit()
        cursor.close()
    except:
        print("Search failed")
        result = [(-1,"")]
        conn.commit()
        cursor.close()    
    return result[0][0], result[0][1]

In [18]:
def update_encoding(conn,id,prompt2,answer):
    sql = "UPDATE experiments SET encoding = '"+answer +"', prompt2 = '"+ prompt2 +"' WHERE id = " + str(id)
    #print(sql)
    cursor = conn.cursor()
    try:
        cursor.execute(sql)
        conn.commit()
        cursor.close()
    except:
        print("Update failed. SQL: ")
        print(sql)
        conn.commit()
        cursor.close()

In [27]:
versions = [10] # done: 9 
for version in versions:
    for project in projects:
        query = dict(project)
        story = query["story"]
        if story in ['creek', 'minotaur','seeknumbers', 'sudoku', 'yosenabe','hop']:
            continue
        for example_1 in projects:
            story_1 = example_1["story"]
            if example_1 == project:
                continue
            for example_2 in projects:
                if example_2 == project or example_2 == example_1:
                    continue                
                story_2 = example_2["story"]
                if story == 'lights' and story_1 == 'creek' and story_2 in ['minotaur', 'seeknumbers']:
                    continue
                id, query['Generation rules'] = search_encoding(conn,story,story_1,story_2,str(version))
                ct = datetime.now()
                print(f"Version: {version}. Story {story}. Context: {story_1} and {story_2}. \t Timestamp: {ct}")
                answer, prompt2 = pipeline_complement(example_1,example_2,query)
                answer = answer.replace("'","")
                prompt2 = prompt2.replace("'","")
                update_encoding(conn,id,prompt2,answer)
                #sys.exit(0)

Version: 10. Story lights. Context: creek and sudoku. 	 Timestamp: 2024-01-01 17:12:44.588738
Version: 10. Story lights. Context: creek and yosenabe. 	 Timestamp: 2024-01-01 17:12:57.860113
Version: 10. Story lights. Context: creek and hop. 	 Timestamp: 2024-01-01 17:13:17.346901
Version: 10. Story lights. Context: minotaur and creek. 	 Timestamp: 2024-01-01 17:13:40.431067
Version: 10. Story lights. Context: minotaur and seeknumbers. 	 Timestamp: 2024-01-01 17:14:17.763006
Version: 10. Story lights. Context: minotaur and sudoku. 	 Timestamp: 2024-01-01 17:14:37.478182
Version: 10. Story lights. Context: minotaur and yosenabe. 	 Timestamp: 2024-01-01 17:14:54.776726
Version: 10. Story lights. Context: minotaur and hop. 	 Timestamp: 2024-01-01 17:15:14.869576
Version: 10. Story lights. Context: seeknumbers and creek. 	 Timestamp: 2024-01-01 17:15:47.435999
Version: 10. Story lights. Context: seeknumbers and minotaur. 	 Timestamp: 2024-01-01 17:22:55.363268
Version: 10. Story lights. Con